# Read trimmer for Next-Generation-Sequencing data

## Description

The advent of Next Generation Sequencing (NGS) technologies have transformed how biological research is being performed and today almost all biological fields use the technology for cutting edge discoveries. Today, a human genome can be sequenced in very short time for approximately $1000 giving unprecedented possibilities for investigating human traits, evolution and diseases. Similarly whole bacterial communities and their interplay with the environment can be studied, unravelling novel enzymes and organisms. These experiments produces massive amounts of data that often including a lot of noise and the program therefore has to be able to clean up NGS data.


### Input and output - fastq format

The input and output data is Illumina fastq files, which is very similar to a fasta file except that they also contain a quality associated to each base so that one entry is exactly four lines which is also know as a 'read':

*Line 1 begins with a '@' character and is followed by a sequence identifier and an optional description (like a FASTA title line).
Line 2 is the raw sequence letters.
Line 3 begins with a '+' character and is optionally followed by the same sequence identifier (and any description) again.
Line 4 encodes the quality values for the sequence in Line 2, and must contain the same number of symbols as letters in the sequence.*

This is an example:
> @ILLUMINA-3BDE4F_0027:2:1:12594:2417#TGACCA/1
    GTTACTTGTGTCGTTGTAGACACTNCTGATACCTCCAGCATGCCTCACAGCACACCTTCGCAGGCTTACAGAACGCTCCCCTACCCAACAACACATAGTGT
    +\
    a\EDZKOGNDPDIJDKFFNF]Z]`BaaaZaZEZVcaaX]accccccccccccccccccccccccc[ccacccccccccccccabcca_c_O____a_aXaV

Each of the base-qualities (fourth line) encodes the probability that the base at the same position is wrong using the ascii table for conversion between the characters and a number (probability). The input data contains millions (and up to billions) of these reads and they must be cleaned according to user specified settings.


## Details

The program must be able to:

- Read both compressed and uncompressed fastq files and be able to write both compressed or uncompressed files according to user input (gzip only).
- Be able to use both Phred+33 and Phred+64 encoding according to user input.
- Autodetect the quality scale of a file (phred+33 or phred+64).
- Trim X nucleotides from the 3' of each read given user input.
- Trim each read from the 3' based on quality, either as minimum (single residue) or mean of moving window.
- These trims from 3' can be done independently of each other in above order.
- Trim X nucleotides from the 5' of each read given user input.
- Trim each read from the 5' based on quality, either as minimum (single residue) or mean of moving window.
- These trims from 5' can be done independently of each other in above order.
- Filter out reads with a mean quality lower than specified after trimming.
- Filter out reads that are shorter than specified after trimming.
- Filter out reads that have a more than a specified number of N bases (unknown bases).
- Must keep track of how many reads are trimmed, removed etc. Put the result in a (user specified) log file.

Optionally advanced options can be implemented into the program:

- Autodetect if input files are gzipped or not.
- Trim reads from the 3' and/or 5' based on quality as minimum of moving window, i.e. if window contains any low quality, trim and move.
- Calculate statistics on a fastq/fasta file without trimming such as number of entries, number of each bases, length of entries, average length of entries, quality (average, best/worst 10%), etc.
NGS files are big, hence they take a long time to process. Optimize the program for performance. Requires lots of testing and rewriting code.
- Trim paired end reads simultaneously keeping pairs in sync between the two files. This complicated the process, see below.

Since NGS files are so big, the resulting program must be efficient in RAM usage and fast due to the millions of reads in real data sets.

The program has to be user friendly, but should only be based on command line execution, i.e. not an interactive user interface. You can use the Python standard library argparse to deal with the many options.


## Paired end reads
When the a long DNA molecule is sequenced from both ends this will yield two files, one with reads from the 5' and one with the reads from the 3' of the DNA molecule. Programs will assume that paired reads will have the same position in the two files. 

When trimming the two files they have to be kept in sync, eg. if a read is removed from one file, then the corresponding pair needs to be removed from the other file to maintain synchronism between the files.

-------------------------------------------------------

## Phred scores

The quality score of a base, also known as a Phred or Q score, is an integer value representing the estimated probability of an error, i.e. that the base is incorrect. 

Q scores are often represented as ASCII characters. Tables converting between integer Q scores, ASCII characters and error probabilities are shown in the table below.

![ASCII code](https://drive5.com/usearch/manual/qscores.gif)


#### Phred+33
To use the Phred+33 encoding, take the Phred quality score, add 33 to it, then use the ASCII character corresponding to the sum. For example, using the Phred+33 encoding, a quality score of 30 would be represented with the ASCII character with the ascii code of 63 (30 + 33), which is ‘?’.

#### Phred+64
The Phred+64 encoding works the same as the Phred+33 encoding, except you add 64 to the Phred score to determine the ASCII code of the quality character. You will only find Phred+64 encoding on older data, which was sequenced several years ago. The tricky part is that there is no indication in the FASTQ file as to which encoding was used, you have to make an educated guess.


### Recognizing the format

A quick way to check visually whether we are working with Phred 33 or 64 is to look for # and $, which means ASCII_BASE=33 or lower-case letters which probably implies ASCII_BASE=64.

----------------------------------------------------

See https://drive5.com/usearch/manual/quality_score.html for explanation on Phred scores.

See https://bioinformaticsworkbook.org/introduction/fastqquality-score-encoding.html#gsc.tab=0 for hint on how to tell Phred 33 from 64 apart.



## Reading compressed data

See https://www.tutorialspoint.com/python-support-for-gzip-files-gzip for information on how to deal with compressed data in Python.

> import gzip\
  with gzip.open('input.gz','r') as infile:        
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for line in infile:        
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;print('got line', line)

## Trimmomatic - an example of an NGS read trimmer software

Full paper can be found at https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4103590/pdf/btu170.pdf 

### Removal of technical sequences (adapters)

Trimmomatic uses two approaches to detect technical sequences within the reads. 
- **Simple mode**\
    It works by finding an approximate match between the read and the user-supplied technical sequence. This mode has the advantage of working for all technical sequences, including adapters and polymerase chain reaction (PCR) primers, or fragments thereof. Such sequences can be detected in any location or orientation within the reads but requires a substantial minimum overlap between the read and technical sequence to prevent false-positive findings. However, short partial adapter sequences, which often occur at the ends of reads, are inherently unable to meet this minimum overlap requirement and therefore are not detectable.
    
- **Palindrome mode**\
    It is specifically aimed at detecting this common ‘adapter read-through’ scenario, whereby the sequenced DNA fragment is shorter than the read length, and results in adapter contamination on the end of the reads. This is especially the case for longer read length as supported by the Miseq. Although such short fragments should normally be removed during library preparation, in practice this process is not perfectly efficient, and thus many libraries suffer from this problem to some extent. ‘Palindrome mode’ can only be used with paired-end data, but has considerable advantages in sensitivity and specificity over ‘simple’ mode.


### Quality filtering

Trimmomatic offers two main quality filtering alternatives. Both approaches exploit the Illumina quality score of each base position to determine where the read should be cut, resulting in the retention of the 5' portion, while the sequence on the 3' of the cut point is discarded. This fits well with typical Illumina data, which generally have poorer quality toward the 3' end. 
- **Simple 5’ and 3’ end-trimmers**\
    These start at the specified end of the read, and remove a successive series of bases which are all below the specified quality score. These trimmers are primarily designed for removal of artifacts, such as ‘N’ base calls or a series of ‘B’ quality scores which were introduced by the 1.3 Illumina pipeline. These modes are not intended for quality filtering, but rather for removing artifacts which may cause sub-optimal trimming by the sliding window or maximum information trimming steps.  
- **Sliding Window quality filtering**\
    The Sliding Window uses a relatively standard approach. This works by scanning from the 5' end of the read, and removes the 3' end of the read when the average quality of a group of bases drops below a specified threshold. This prevents a single weak base causing the removal of subsequent high-quality data, while still ensuring that a consecutive series of poor-quality bases will trigger trimming.
       
- **Maximum Information quality filtering**\
    For many applications, the incremental value of retaining additional bases in a read is related to the read length. *Short reads are almost worthless* because they occur multiple times within the target sequence and thus they give only ambiguous information. Even at the risk of introducing errors, it is worthwhile to retain additional low-quality bases early in a read, so that the trimmed read is sufficiently long to be informative.\
    However, *beyond a certain read length, retaining additional bases is less beneficial*, and may even be detrimental. Reads of moderate length are likely to be already informative and, depending on the task at hand, can be almost as valuable as fulllength reads. Therefore, the smaller potential benefit of retaining additional bases must be balanced against the increasing risk of retaining errors, which could cause the existing read value to be lost.\
    As such, *it is worthwhile for the trimming process to become increasingly strict as it progresses through the read*, rather than to apply a fixed quality threshold. 


### Read Filtering
Read filtering drops a read entirely if it does not meet the speci-fied criteria. Currently supported criteria are ‘MINLEN’, which requires that a read be of a specified minimal length, and ‘AVGQUAL’ which requires that the read have a specified aver-age quality across all bases. 


### Quality Score Conversion
Quality score conversion simplifies pipelines which have to deal with a combination of Phred+33 and Phred+64 data, allowing downstream tools to treat data uniformly. Conversion is supported in both directions. 

        
### Implementation
Trimmomatic uses a pipeline-based architecture, allowing individual ‘steps’ (adapter removal, quality filtering, etc.) to be applied to each read/read pair, in the order specified by the user. Each step can choose to work on the reads in isolation, or work on the combined pair, as appropriate. The tool tracks read pairing and stores ‘paired’ and ‘single’ reads separately.
    
